In [13]:
# Libraries for feature extraction:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pathlib
from scipy.stats import kurtosis
from scipy.stats import skew


In [14]:
#Audio data directory:
audio_dir = './Data/genres/'
test_dir = './Data/test_songs/'

In [15]:
#Dictionary for genres label encoding:
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [16]:
def split_songs(X, window = 0.2, overlap = 0.5):
    """
    Function to split a song into multiple songs.
    """
    # Temporary lists to hold results
    temp_X = []

    # Get input song array size
    xshape = X.shape[0]
    chunk = int(xshape*window)
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones
    spsong = [X[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)

    return np.array(temp_X)

In [17]:
#Get selected features from each song using librosa and numpy:

def get_features(y, sr, n_fft = 1024, hop_length = 512):
    # Selected features:
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None, 'zcr': None, 'chroma':None}
    
    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rmse'] = librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    
    # Treatment of MFCC feature
    mfcc = librosa.feature.mfcc(y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Calculate statistics for each feature:
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result[f'{k}_mean'] = np.mean(v)
            result[f'{k}_std'] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
    
    #Calculating one more feature:
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [18]:

def read_process_songs(src_dir, debug = True):
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
    
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)
                signal = signal[:660000]
                
                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))
                    
                #Split songs:
                samples=split_songs(signal)

                # Append the result to the data structure
                for s in samples:
                    features = get_features(s, sr)
                    features['genre'] = genres[x]
                    arr_features.append(features)
    return arr_features

In [30]:
def read_process_songs_t3(src_dir, debug = True):
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
    
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)
                samples=[(signal[:220000],sr),(signal[220000:440000],sr),(signal[440000:],sr)]

                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))

                # Append the result to the data structure
                for s in samples:
                    features = get_features(s[0], s[1])
                    features['genre'] = genres[x]
                    arr_features.append(features)
    return arr_features

In [31]:
def read_process_songs_t1(src_dir, debug = True):
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
    
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)

                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))

                # Append the result to the data structure
                features = get_features(signal, sr)
                features['genre'] = genres[x]
                arr_features.append(features)
    return arr_features

In [26]:
# Get list of dicts with features and convert to dataframe
features = read_process_songs(audio_dir,debug=True)

In [20]:
df=pd.DataFrame(features)

In [25]:
df.to_csv('./Features/features_augmented.csv', index=False)

In [33]:
features1 = read_process_songs_t1(test_dir,debug=True)
df1=pd.DataFrame(features1)
df1.to_csv('./Features/features_augmented_t1.csv', index=False)

Reading file: ./Data/test_songs/blues/blues.00099.wav
Reading file: ./Data/test_songs/classical/classical.00099.wav
Reading file: ./Data/test_songs/country/country.00099.wav
Reading file: ./Data/test_songs/disco/disco.00099.wav
Reading file: ./Data/test_songs/hiphop/hiphop.00099.wav
Reading file: ./Data/test_songs/jazz/jazz.00099.wav
Reading file: ./Data/test_songs/metal/metal.00099.wav
Reading file: ./Data/test_songs/pop/pop.00099.wav
Reading file: ./Data/test_songs/reggae/reggae.00099.wav
Reading file: ./Data/test_songs/rock/rock.00099.wav


In [34]:
features3 = read_process_songs_t3(test_dir,debug=True)
df3=pd.DataFrame(features3)
df3.to_csv('./Features/features_augmented_t3.csv', index=False)

Reading file: ./Data/test_songs/blues/blues.00099.wav
Reading file: ./Data/test_songs/classical/classical.00099.wav
Reading file: ./Data/test_songs/country/country.00099.wav
Reading file: ./Data/test_songs/disco/disco.00099.wav
Reading file: ./Data/test_songs/hiphop/hiphop.00099.wav
Reading file: ./Data/test_songs/jazz/jazz.00099.wav
Reading file: ./Data/test_songs/metal/metal.00099.wav
Reading file: ./Data/test_songs/pop/pop.00099.wav
Reading file: ./Data/test_songs/reggae/reggae.00099.wav
Reading file: ./Data/test_songs/rock/rock.00099.wav
